# # AI Workshop - Lab 3-1b: Scheduling
## Employee Scheduling Optimization Using Google OR-Tools

This notebook will guide you through solving an employee scheduling problem using Google OR-Tools. We will schedule employees for a workweek, ensuring each employee works within their required minimum and maximum hours, while also meeting a minimum staffing level for each shift.

## Problem Overview

Imagine a company that needs to schedule employees for a workweek. Each employee has specific constraints:
- **Minimum shifts**: The least number of shifts they must work during the week.
- **Maximum shifts**: The most number of shifts they can work during the week.

Additionally, the company requires a minimum number of employees to be working during each shift.

The goal is to determine the optimal schedule that meets all these constraints.

In [ ]:
!pip install ortools

In [ ]:
from ortools.linear_solver import pywraplp

## Problem Data

We'll use the following data for our problem:

- **Employees**: A list of employees, each with their own minimum and maximum shifts.
- **Shifts**: A list of shifts (e.g., morning, afternoon, evening) for each day of the workweek.
- **Minimum Staffing**: The minimum number of employees required for each shift.

Fill out this data with your own values to test the model.

In [ ]:
# Data

# Employees and their constraints
employees = {
    'Alice': {'min_shifts': 2, 'max_shifts': 14},
    'Bob': {'min_shifts': 3, 'max_shifts': 15},
    'Charlie': {'min_shifts': 2, 'max_shifts': 13},
    'David': {'min_shifts': 4, 'max_shifts': 5},
    'Eve': {'min_shifts': 3, 'max_shifts': 14},
    'Frank': {'min_shifts': 2, 'max_shifts': 3},
}

# Shifts for one week (Monday to Friday, 3 shifts per day)
shifts = [
    'Monday Morning', 'Monday Afternoon', 'Monday Evening',
    'Tuesday Morning', 'Tuesday Afternoon', 'Tuesday Evening',
    'Wednesday Morning', 'Wednesday Afternoon', 'Wednesday Evening',
    'Thursday Morning', 'Thursday Afternoon', 'Thursday Evening',
    'Friday Morning', 'Friday Afternoon', 'Friday Evening',
]

# Minimum number of employees required per shift
min_staffing = {
    shift: 2 for shift in shifts  # All shifts require at least 2 employees
}

# Verify data completeness
assert len(employees) > 0, "No employees defined"
assert len(shifts) > 0, "No shifts defined"
assert all(shift in min_staffing for shift in shifts), "Incomplete staffing requirements"

## Model

We'll define an integer programming model for this problem. Each employee will have a binary variable for each shift, indicating whether they are assigned to that shift. We will add constraints to ensure:
1. Each employee's total hours fall within their minimum and maximum limits.
2. Each shift has at least the required minimum number of employees.

In [ ]:
# Create the solver
solver = pywraplp.Solver.CreateSolver('SCIP')  # SCIP is suitable for integer programming

## Decision Variables

We'll create binary decision variables to represent whether an employee is assigned to a shift. These variables will be 1 if the employee works a shift, and 0 otherwise.

In [ ]:
# Decision variables
variables = {}
for employee in employees:
    for shift in shifts:
        variables[employee, shift] = solver.BoolVar(f'{employee}_{shift}')

# Verify variables
len(variables)

## Constraints

### 1. Employee Hour Constraints

Each employee must work within their specified minimum and maximum shifts during the week.

### 2. Minimum Staffing Constraints

Each shift must have at least the required number of employees assigned to it.

In [ ]:
# Constraints

# Employee Hour Constraints: each employee must work between their minimum and maximum shifts
for employee, constraints in employees.items():
    total_shifts = sum(variables[employee, shift] for shift in shifts)
    solver.Add(total_shifts >= constraints['min_shifts'])
    solver.Add(total_shifts <= constraints['max_shifts'])

# Minimum Staffing Constraints: each shift must have at least the required number of employees
for shift in shifts:
    total_staffing = sum(variables[employee, shift] for employee in employees)
    solver.Add(total_staffing >= min_staffing[shift])

# Verify constraints
solver.NumConstraints()

## Objective Function

The goal is to balance the workload as evenly as possible among employees. We'll minimize the maximum number of hours worked by any single employee to achieve this.

In [ ]:
# Objective: Minimize the maximum workload
max_workload = solver.NumVar(0, solver.infinity(), 'max_workload')

for employee in employees:
    total_shifts = sum(variables[employee, shift] for shift in shifts)
    solver.Add(total_shifts <= max_workload)

solver.Minimize(max_workload)

## Solve the Model

Let's run the solver and check if it finds an optimal solution.

In [ ]:
# Solve the model
status = solver.Solve()

# Check if the problem has an optimal solution
if status == pywraplp.Solver.OPTIMAL:
    print('Optimal Solution Found')
else:
    print('The problem does not have an optimal solution')


## Results

If an optimal solution is found, we can extract the schedule for each employee and check the staffing levels for each shift.

In [ ]:
# Print the schedule
if status == pywraplp.Solver.OPTIMAL:
    print('Employee Schedule:')
    for employee in employees:
        shifts_worked = [
            shift for shift in shifts if variables[employee, shift].solution_value() > 0.5
        ]
        print(f'{employee}: {shifts_worked}')

    print('\nStaffing Levels:')
    for shift in shifts:
        staff = [
            employee for employee in employees if variables[employee, shift].solution_value() > 0.5
        ]
        print(f'{shift}: {len(staff)} employees')
else:
    print('No feasible schedule found')


# Extending the Employee Scheduling Model: Minimizing Cost

In this extension, we'll introduce a different rate of pay for each employee. Our goal is to minimize the total cost of the schedule, in addition to meeting all the constraints we've already defined. Each employee has a specified hourly rate, and the total cost will depend on the number of shifts they work during the week.

---

## Adding Employee Pay Rates

Each employee is assigned a pay rate. These rates will be multiplied by the shifts worked to compute the total cost. We'll add this to our model by modifying the objective function.

Here's how we'll do it:
1. **Define Pay Rates**: Add a dictionary that specifies the shift rate for each employee.
2. **Modify the Objective Function**: Update the objective function to minimize the total pay instead of balancing workload.
3. **Guide the Participant**: They'll implement this step-by-step with hints.



In [ ]:
# Pay rates for each employee
pay_rates = {
    'Alice': 25,  # $25/shift
    'Bob': 30,    # $30/shift
    'Charlie': 20 # $20/shift
}

# Verify the pay rates match the employees
assert set(pay_rates.keys()) == set(employees.keys()), "Pay rates must be defined for all employees"

### Modifying the Objective Function

We need to modify the model to minimize the total cost. The total cost is calculated as:
$$
\text{Total Cost} = \sum_{\text{employee}} (\text{hours worked by employee} \times \text{pay rate})
$$

---

To include this in the model:
1. For each employee, compute the total hours worked by summing their assigned shifts.
2. Multiply the total hours by their hourly pay rate.
3. Add this term to the solver's objective using the `SetCoefficient` method.

In [ ]:
# Step 1: Create the objective function
total_cost = solver.Objective()

# Step 2: Add cost contributions from each employee
for employee in employees:
    total_shifts = sum(variables[employee, shift] for shift in shifts)
    # Multiply total hours by the pay rate and add to the objective
    total_cost.SetCoefficient(total_shifts, pay_rates[employee])

# Step 3: Set the objective to minimize total cost
total_cost.SetMinimization()

### Solve and Compare

Now that we've updated the objective, let's re-run the model to find the new optimal solution. The constraints remain the same, but the solution will now prioritize minimizing costs over balancing workloads.


In [ ]:
# Solve the modified model
status = solver.Solve()

# Check if the problem has an optimal solution
if status == pywraplp.Solver.OPTIMAL:
    print('Optimal Solution Found')
else:
    print('The problem does not have an optimal solution')

In [ ]:
# Print the cost-optimized schedule and total cost
if status == pywraplp.Solver.OPTIMAL:
    print('Cost-Optimized Schedule:')
    total_cost = 0
    for employee in employees:
        shifts_worked = [
            shift for shift in shifts if variables[employee, shift].solution_value() > 0.5
        ]
        employee_shifts = len(shifts_worked)
        employee_cost = employee_hours * pay_rates[employee]
        total_cost += employee_cost
        print(f'{employee}: {shifts_worked} (Shifts: {employee_shifts}, Cost: ${employee_cost:.2f})')

    print(f'\nTotal Cost: ${total_cost:.2f}')
else:
    print('No feasible schedule found')


---

### Final Questions:

1. **Trade-offs in Objectives**: How does the cost-optimized schedule compare to the workload-balanced schedule? Are there significant differences in hours worked or staffing levels?
2. **Experiment with Pay Rates**: What happens if you increase Alice's pay rate to $50/hour? Does the model still assign her the same number of shifts?
3. **Hybrid Objective**: Can you think of a way to balance both minimizing costs and workloads? Hint: You might combine the two objectives into a weighted sum.

This extension demonstrates the flexibility of linear programming models and the power of Google OR-Tools for tackling real-world scheduling problems with multiple objectives.
